# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'capstoneproject'

experiment=Experiment(ws, experiment_name)

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'compute-jupyter'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


In [5]:
from azureml.core import Workspace, Dataset

from sklearn.model_selection import train_test_split
subscription_id = 'baa67dbf-45d0-4d84-b662-527186361068'
resource_group = 'dwtr-t332-20210421'
workspace_name = 'DSStudio'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='BankMarketing')
df=dataset.to_pandas_dataframe()
df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":30, "enable_early_stopping":True, "n_cross_validations":3,"max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT',**automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on compute-jupyter with default configuration
Running on remote compute: compute-jupyter


Experiment,Id,Type,Status,Details Page,Docs Page
capstoneproject,AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstoneproject,AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7',
 'target': 'compute-jupyter',
 'status': 'Completed',
 'startTimeUtc': '2022-05-21T14:21:58.139662Z',
 'endTimeUtc': '2022-05-21T14:39:07.846054Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'compute-jupyter',
  'AMLSettingsJsonString': '{"path":null,"name":"capstoneproject","subscription_id":"baa67dbf-45d0-4d84-b662-527186361068","resource_group":"dwtr-t332-20210421","workspace_name":"DSStudio","region":"eastus2","compute_target":"compute-jupyter","spark_service

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [36]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: capstoneproject,
Id: AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7_36,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/jobs/...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('25', Pipeline(memory=None, steps=[('robustscaler', RobustScaler(copy=True, quantile_range=[10, 90], with_cen

In [35]:
best_run_metrics

{'AUC_weighted': 0.9297147142022917,
 'AUC_macro': 0.9297147142022918,
 'AUC_micro': 0.9367216080945812,
 'accuracy': 0.8929729729729731,
 'average_precision_score_weighted': 0.935392710543689,
 'precision_score_micro': 0.8929729729729731,
 'norm_macro_recall': 0.7361428874782913,
 'f1_score_weighted': 0.8918197778197778,
 'precision_score_macro': 0.8870429269193122,
 'precision_score_weighted': 0.8933046772262458,
 'f1_score_macro': 0.875851322590453,
 'recall_score_macro': 0.8680714437391456,
 'f1_score_micro': 0.8929729729729731,
 'weighted_accuracy': 0.9125144028346148,
 'recall_score_weighted': 0.8929729729729731,
 'recall_score_micro': 0.8929729729729731,
 'matthews_correlation': 0.7546911982185293,
 'average_precision_score_macro': 0.9195895708449164,
 'average_precision_score_micro': 0.9385993314835198,
 'log_loss': 0.35616470485146784,
 'balanced_accuracy': 0.8680714437391456,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7

In [69]:
print("Best AutoML model Accuracy: ", best_run_metrics['accuracy'])

Best AutoML model Accuracy:  0.8929729729729731


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [61]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model.steps)

Run(Experiment: capstoneproject,
Id: AutoML_e8ea9e9e-8768-44c2-bcf1-8165282894b7_36,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('25', Pipeline(memory=None, steps=[('robustscaler', RobustScaler(copy=True, quantile_range=[10, 90], with_centering=False, with_scaling=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='goss', colsa

In [62]:
model = best_run.register_model(model_name='automl_model',
                           model_path='outputs/model.pkl',
                           tags=best_run.get_metrics())
print(model.name, model.id, model.version, sep='\t')

model	model:2	2


In [63]:
import joblib
joblib.dump(fitted_model, 'automl-votingEnsemble_model.joblib')

['automl-votingEnsemble_model.joblib']

Automl model registered with Accuracy 0.89
